# Detección de Fraude con Aprendizaje Semi-Supervisado

## Pseudocódigo Formal del Algoritmo Self-Training

**ALGORITMO** `SelfTraining(D_labeled, D_unlabeled, threshold)`

**ENTRADA:**
- `D_labeled ← {(x₁, y₁), (x₂, y₂), …, (xₙ, yₙ)}` (conjunto de datos etiquetados)
- `D_unlabeled ← {x₁', x₂', …, xₘ'}` (conjunto de datos sin etiquetas)
- `threshold` (umbral de confianza para pseudo-etiquetado)

**SALIDA:**
- Modelo entrenado `M_final`
- Conjunto expandido `D_expanded` con pseudo-etiquetas

**PASOS:**

1. **Inicialización:**
   - `M ← entrenar_modelo_base(D_labeled)`
   - `D_temp ← D_labeled`
   - `D_remaining ← D_unlabeled`

2. **MIENTRAS** `D_remaining ≠ ∅` **Y** condición de parada no cumplida:
   
   a. **Predecir con confianza:**
      - Para cada `xᵢ ∈ D_remaining`:
        - `prob_i ← M.predict_proba(xᵢ)`
        - `confidence_i ← MAX(prob_i)`
   
   b. **Seleccionar predicciones de alta confianza:**
      - `D_confident ← {(xᵢ, ŷᵢ) | confidence_i ≥ threshold}`
      - Donde `ŷᵢ ← argmax(prob_i)`
   
   c. **SI** `D_confident = ∅` **ENTONCES**:
      - **ROMPER** (no hay predicciones confiables)
   
   d. **Expandir conjunto etiquetado:**
      - `D_temp ← D_temp ∪ D_confident`
      - `D_remaining ← D_remaining \ D_confident`
   
   e. **Re-entrenar modelo:**
      - `M ← entrenar_modelo(D_temp)`

3. **RETORNAR** modelo final `M` y conjunto expandido `D_temp`



## 1. Importacion de Librerias y Carga de Datos

In [1]:
# Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# Cargar datos
datos_entrenamiento = pd.read_csv("application_train.csv")
print(f"Datos: {datos_entrenamiento.shape} | Fraude: {datos_entrenamiento['TARGET'].mean()*100:.1f}%")
datos_entrenamiento.head()

Datos: (307511, 122) | Fraude: 8.1%


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Preprocesamiento de Datos

In [2]:
# Seleccionar variables numericas
datos_numericos = datos_entrenamiento.select_dtypes(include=[np.number])
print(f"Variables numericas: {datos_numericos.shape[1]}")

Variables numericas: 106


In [3]:
# Seleccionar variables categoricas
datos_categoricos = datos_entrenamiento.select_dtypes(include=['object'])
print(f"Categoricas: {datos_categoricos.shape[1]}")

Categoricas: 16


In [4]:
# Renombrar columnas a español
nombres_espanol = {
    'TARGET': 'objetivo',
    'CNT_CHILDREN': 'num_hijos',
    'CNT_FAM_MEMBERS': 'num_miembros_familia',
    'AMT_INCOME_TOTAL': 'ingreso_total',
    'AMT_CREDIT': 'monto_credito',
    'AMT_ANNUITY': 'anualidad',
    'AMT_GOODS_PRICE': 'precio_bienes',
    'DAYS_BIRTH': 'edad_dias',
    'DAYS_EMPLOYED': 'dias_empleado',
    'DAYS_REGISTRATION': 'dias_registro',
    'DAYS_ID_PUBLISH': 'dias_publicacion_id',
    'OWN_CAR_AGE': 'edad_auto',
    'EXT_SOURCE_1': 'score_externo_1',
    'EXT_SOURCE_2': 'score_externo_2',
    'EXT_SOURCE_3': 'score_externo_3',
    'REGION_POPULATION_RELATIVE': 'poblacion_region',
    'REGION_RATING_CLIENT': 'calificacion_region',
    'REGION_RATING_CLIENT_W_CITY': 'calificacion_region_ciudad',
    'FLAG_MOBIL': 'tiene_movil',
    'FLAG_EMP_PHONE': 'tiene_telefono_trabajo',
    'FLAG_WORK_PHONE': 'tiene_telefono_laboral',
    'FLAG_CONT_MOBILE': 'movil_contactable',
    'FLAG_PHONE': 'tiene_telefono',
    'FLAG_EMAIL': 'tiene_email',
    'HOUR_APPR_PROCESS_START': 'hora_solicitud',
    'OBS_30_CNT_SOCIAL_CIRCLE': 'obs_30dias_circulo_social',
    'DEF_30_CNT_SOCIAL_CIRCLE': 'def_30dias_circulo_social',
    'OBS_60_CNT_SOCIAL_CIRCLE': 'obs_60dias_circulo_social',
    'DEF_60_CNT_SOCIAL_CIRCLE': 'def_60dias_circulo_social',
    'DAYS_LAST_PHONE_CHANGE': 'dias_ultimo_cambio_telefono',
    'AMT_REQ_CREDIT_BUREAU_HOUR': 'consultas_buro_ultima_hora',
    'AMT_REQ_CREDIT_BUREAU_DAY': 'consultas_buro_ultimo_dia',
    'AMT_REQ_CREDIT_BUREAU_WEEK': 'consultas_buro_ultima_semana',
    'AMT_REQ_CREDIT_BUREAU_MON': 'consultas_buro_ultimo_mes',
    'AMT_REQ_CREDIT_BUREAU_QRT': 'consultas_buro_ultimo_trimestre',
    'AMT_REQ_CREDIT_BUREAU_YEAR': 'consultas_buro_ultimo_ano',
    'NAME_CONTRACT_TYPE': 'tipo_contrato',
    'CODE_GENDER': 'genero',
    'FLAG_OWN_CAR': 'tiene_auto',
    'FLAG_OWN_REALTY': 'tiene_propiedad',
    'NAME_TYPE_SUITE': 'tipo_acompanante',
    'NAME_INCOME_TYPE': 'tipo_ingreso',
    'NAME_EDUCATION_TYPE': 'nivel_educacion',
    'NAME_FAMILY_STATUS': 'estado_civil',
    'NAME_HOUSING_TYPE': 'tipo_vivienda',
    'OCCUPATION_TYPE': 'ocupacion',
    'WEEKDAY_APPR_PROCESS_START': 'dia_semana_solicitud',
    'ORGANIZATION_TYPE': 'tipo_organizacion',
    'FONDKAPREMONT_MODE': 'modo_fondo_reparacion',
    'HOUSETYPE_MODE': 'modo_tipo_casa',
    'WALLSMATERIAL_MODE': 'modo_material_paredes',
    'EMERGENCYSTATE_MODE': 'modo_estado_emergencia'
}

datos_entrenamiento = datos_entrenamiento.rename(columns=nombres_espanol)
datos_numericos = datos_entrenamiento.select_dtypes(include=[np.number])
datos_categoricos = datos_entrenamiento.select_dtypes(include=['object'])
print(f"Dataset renombrado: {datos_numericos.shape[1]} numericas + {datos_categoricos.shape[1]} categoricas")

Dataset renombrado: 106 numericas + 16 categoricas


In [5]:
# Imputar nulos numericos con mediana
datos_numericos = datos_numericos.fillna(datos_numericos.median())
print("Nulos numericos imputados")

Nulos numericos imputados


In [6]:
# Imputar nulos categoricos con moda
datos_categoricos = datos_categoricos.fillna(datos_categoricos.mode().iloc[0])
print("Nulos categoricos imputados")

Nulos categoricos imputados


In [7]:
# Codificar variables categoricas a numeros
from sklearn.preprocessing import LabelEncoder

columnas_categoricas = datos_entrenamiento.select_dtypes(include=['object']).columns
datos_categoricos_codificados = datos_categoricos.copy()

for columna in columnas_categoricas:
    le = LabelEncoder()
    datos_categoricos_codificados[columna] = le.fit_transform(datos_categoricos_codificados[columna].astype(str))

print(f"Codificadas: {len(columnas_categoricas)} variables")
datos_categoricos_codificados.head()

Codificadas: 16 variables


,tipo_contrato,genero,tiene_auto,tiene_propiedad,tipo_acompanante,tipo_ingreso,nivel_educacion,estado_civil,tipo_vivienda,ocupacion,dia_semana_solicitud,tipo_organizacion,modo_fondo_reparacion,modo_tipo_casa,modo_material_paredes,modo_estado_emergencia
0,0,1,0,1,6,7,4,3,1,8,6,5,2,0,5,0
1,0,0,0,0,1,4,1,1,1,3,1,39,2,0,0,0
2,1,1,1,1,6,7,4,3,1,8,1,11,2,0,4,0
3,0,0,0,1,6,7,4,0,1,8,6,5,2,0,4,0
4,0,1,0,1,6,7,4,3,1,3,4,37,2,0,4,0


In [8]:
# Mapeo de codigos a categorias originales
mapeo_codigos = {}

for columna in columnas_categoricas:
    le = LabelEncoder()
    valores_unicos = datos_categoricos[columna].unique()
    le.fit(valores_unicos.astype(str))
    mapeo_codigos[columna] = dict(zip(range(len(le.classes_)), le.classes_))

print("Mapeo de codigos:\n")
for columna, mapeo in mapeo_codigos.items():
    print(f"{columna}:")
    for codigo, categoria in sorted(mapeo.items()):
        print(f"  {codigo} → {categoria}")
    print()

Mapeo de codigos:

tipo_contrato:
  0 → Cash loans
  1 → Revolving loans

genero:
  0 → F
  1 → M
  2 → XNA

tiene_auto:
  0 → N
  1 → Y

tiene_propiedad:
  0 → N
  1 → Y

tipo_acompanante:
  0 → Children
  1 → Family
  2 → Group of people
  3 → Other_A
  4 → Other_B
  5 → Spouse, partner
  6 → Unaccompanied

tipo_ingreso:
  0 → Businessman
  1 → Commercial associate
  2 → Maternity leave
  3 → Pensioner
  4 → State servant
  5 → Student
  6 → Unemployed
  7 → Working

nivel_educacion:
  0 → Academic degree
  1 → Higher education
  2 → Incomplete higher
  3 → Lower secondary
  4 → Secondary / secondary special

estado_civil:
  0 → Civil marriage
  1 → Married
  2 → Separated
  3 → Single / not married
  4 → Unknown
  5 → Widow

tipo_vivienda:
  0 → Co-op apartment
  1 → House / apartment
  2 → Municipal apartment
  3 → Office apartment
  4 → Rented apartment
  5 → With parents

ocupacion:
  0 → Accountants
  1 → Cleaning staff
  2 → Cooking staff
  3 → Core staff
  4 → Drivers
  5 → HR

In [9]:
# Combinar datos
datos_completos = pd.concat([datos_numericos, datos_categoricos_codificados], axis=1)
print(f"Dataset final: {datos_completos.shape}")

Dataset final: (307511, 122)
